# Using `eval` for code generation

## Running (evaluating / executing) pieces of code


Once we have a piece of code that we have created, e.g. by modifying a previous piece of code or by creating it from scratch, we want to be able to **run** (**evaluate** / **execute**) it, as if we had typed it straight in.

We can run it at global scope using `eval` (short for "evaluate"):

In [1]:
code = :(x = 3)

:(x = 3)

In [2]:
x

LoadError: UndefVarError: x not defined

In [3]:
eval(code)

3

In [4]:
x

3

The code `x = 3` was run and had the desired **side effect**, namely that a global variable `x` was created and bound to the value `3`. 

# Generating "copy-pasted" code

One important use-case for metaprogramming is generating repetitive code, e.g. for making **wrapper types** and for calling external C libraries.

For example, let's think about a simple type that wraps a floating-point number and counts how many times it gets used during a calculation.

In [5]:
mutable struct MyFloat
    value::Float64
    count::Int
end

In [6]:
MyFloat(x) = MyFloat(x, 0)

MyFloat

We can define the sum of two such numbers:

In [7]:
function Base.:+(x::MyFloat, y::MyFloat)
    x.count += 1
    y.count += 1
    MyFloat(x.value + y.value, 0)
end

In [8]:
x, y = MyFloat(1.0), MyFloat(2.0)

(MyFloat(1.0, 0), MyFloat(2.0, 0))

In [9]:
z = x + y

MyFloat(3.0, 0)

Now we would like to do the same for `-`, `*` and `/`. We could just copy and paste the code 

Copying and pasting is possible but inefficient and prone to error -- you need to remember to replace the function in both places.

First mock out the code:

In [10]:
op = :+

ex = quote
    Base.$(op)(x::MyFloat, y::MyFloat)
        x.count += 1
        y.count += 1

        return MyFloat( ($op)(x.value, y.value), 0)
    end

Base.remove_linenums!(ex)

quote
    Base.:+(x::MyFloat, y::MyFloat)
    x.count += 1
    y.count += 1
    return MyFloat(x.value + y.value, 0)
end

In [11]:
ex

quote
    Base.:+(x::MyFloat, y::MyFloat)
    x.count += 1
    y.count += 1
    return MyFloat(x.value + y.value, 0)
end

In [15]:
for op ∈ (:+, :-, :*, :/)
    @eval function Base.$(op)(x::MyFloat, y::MyFloat)
        x.count += 1
        y.count += 1
        MyFloat(($op)(x.value, y.value), 0)
    end
end

In [16]:
x - y

MyFloat(-1.0, 0)

In [17]:
x * y

MyFloat(2.0, 0)

In [18]:
x

MyFloat(1.0, 3)

# World-age errors

The ability to generate code on the fly is a powerful one, but there is an issue that you may come across: the infamous **world-age errors**; see [this video by Julia Belyakova](https://www.youtube.com/watch?v=d6lTCnhdbqE) for a very good explanation.

This occurs when `eval` is used inside a function, say `f`, to create a new function, say `g`, and then the user tries to call `g` from within `f`. Although it seems like this should be valid, Julia complains:

In [19]:
function f(n)
    new_fun = :(g(x) = 2x)
    eval(new_fun)
    # and now invoke the generated new_fun (g)
    g(n)
end

f (generic function with 1 method)

In [20]:
f(10)

LoadError: MethodError: no method matching g(::Int64)
The applicable method may be too new: running in world age 29650, while current world is 29651.
[0mClosest candidates are:
[0m  g(::Any) at In[19]:2 (method too new to be called from this world context.)

[Sometimes you will not even get a world-age error; Julia will just complain that there is no such function or method.]

## Solving world-age problems

There are (at least) three possible solutions:

### Make sure to return to "top level" before calling `g`:

In [21]:
function f() 
    new_fun = :(g(x) = 2x)
    eval(new_fun)
end

f (generic function with 2 methods)

In [22]:
f()

g (generic function with 1 method)

In [23]:
@time g(3)

  0.000000 seconds


6

###  Use `Base.invokelatest` 

Use `Base.invokelatest` to insist to Julia that it bypasses this world-age mechanism. This comes with a performance cost:

In [24]:
function f(n)
    new_fun = :(g(x) = 2x)
    eval(new_fun)
    Base.@invokelatest g(n)  # or Base.invokelatest(g, n)
end

f (generic function with 2 methods)

In [25]:
f(3)

6

In [26]:
@time f(3)

  0.004402 seconds (847 allocations: 59.998 KiB, 82.98% compilation time)


6

###  Replace `eval` with generated functions

Replace `eval` with https://github.com/JuliaStaging/GeneralizedGenerated.jl or https://github.com/SciML/RuntimeGeneratedFunctions.jl instead:

In [28]:
# using Pkg; Pkg.add("RuntimeGeneratedFunctions")

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed RuntimeGeneratedFunctions ─ v0.5.3
   Installed ExprTools ───────────────── v0.1.6
    Updating `~/Projects/ML_DL/Notebooks/julia-notebooks/JuliaCon2021/Metaprogramming/Project.toml`
  [7e49a35a] + RuntimeGeneratedFunctions v0.5.3
    Updating `~/Projects/ML_DL/Notebooks/julia-notebooks/JuliaCon2021/Metaprogramming/Manifest.toml`
  [e2ba6199] + ExprTools v0.1.6
  [7e49a35a] + RuntimeGeneratedFunctions v0.5.3
Precompiling project...
  ✓ ExprTools
  ✓ RuntimeGeneratedFunctions
  2 dependencies successfully precompiled in 2 seconds (79 already precompiled)


In [29]:
using RuntimeGeneratedFunctions

RuntimeGeneratedFunctions.init(@__MODULE__)

In [30]:
function generate(n)
    new_fun = :(g(x) = $n*x)
    @RuntimeGeneratedFunction(new_fun)
end

generate (generic function with 1 method)

In [31]:
function f(n)
    g = generate(n)
    @time g(3)
    @time g(3)
end

f (generic function with 2 methods)

In [32]:
f(5)

  0.046113 seconds (62.76 k allocations: 3.935 MiB, 99.97% compilation time)
  0.000003 seconds


15

In [33]:
f(3)

  0.003885 seconds (2.75 k allocations: 185.661 KiB, 99.66% compilation time)
  0.000002 seconds


9

In [34]:
@time f(3)

  0.000002 seconds
  0.000002 seconds
  0.000207 seconds (131 allocations: 7.266 KiB)


9